In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import pandas as pd
import os, sys
import cv2
import PIL
import time
from skimage import color
import os.path
from PIL import Image, ImageOps

In [2]:
cwd = os.getcwd()
# os.listdir(cwd)

In [3]:
training_paths = ["./train_tumor/Tumor/", "./train_tumor3/NoTumor/"]
test_paths = ["./test_tumor/Tumor/", "./test_tumor3/NoTumor/"]

In [4]:
# Resizing an saving the training images in same folder with same file name

for path in training_paths:
    for img_name in os.listdir(path):
        f_img = path+img_name
        img = Image.open(f_img)
        #img_gray = ImageOps.grayscale(img)
        img_gray = img.convert('L')
        try:
            img_gray = img_gray.resize((250,250), PIL.Image.ANTIALIAS)
            img_gray.save(f_img, quality=95)
        except ValueError:
            print("There is something wrong")

In [5]:
# Resizing an saving the test images in same folder with same file name

for path in test_paths:
    for img_name in os.listdir(path):
        f_img = path+img_name
        img = Image.open(f_img)
        #img_gray = ImageOps.grayscale(img)
        img_gray = img.convert('L')
        try:
            img_gray = img_gray.resize((250,250), PIL.Image.ANTIALIAS)
            img_gray.save(f_img, quality=95)
        except ValueError:
            print("There is something wrong")

In [6]:
train = ImageDataGenerator(rescale=1/255)
test = ImageDataGenerator(rescale=1/255)

train_dataset = train.flow_from_directory("train_tumor3",
                                          target_size=(250,250),
                                          batch_size = 32,
                                          class_mode = 'binary', color_mode="grayscale")
                                         

test_dataset = test.flow_from_directory("test_tumor3",
                                          target_size=(250,250),
                                          batch_size =32,
                                          class_mode = 'binary', color_mode="grayscale")

Found 88 images belonging to 2 classes.
Found 22 images belonging to 2 classes.


In [7]:
test_dataset.class_indices

{'NoTumor': 0, 'Tumor': 1}

In [8]:
train_dataset.class_indices

{'NoTumor': 0, 'Tumor': 1}

In [9]:
model = keras.Sequential()

# Convolutional layer and maxpool layer 1
model.add(keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(250,250,1)))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 2
model.add(keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 3
model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 4
model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

# This layer flattens the resulting image array to 1D array
model.add(keras.layers.Flatten())

# Hidden layer with 512 neurons and Rectified Linear Unit activation function 
model.add(keras.layers.Dense(512,activation='relu'))

# Output layer with single neuron which gives 0 for Cat or 1 for Dog 
#Here we use sigmoid activation function which makes our model output to lie between 0 and 1
model.add(keras.layers.Dense(1,activation='sigmoid'))

In [10]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

#steps_per_epoch = train_imagesize/batch_size

start = time.time()

model.fit(train_dataset, epochs = 30, batch_size=10, validation_data = 0.0
)

end = time.time() print(end - start)

In [11]:
#steps_per_epoch = train_imagesize/batch_size
result = 0
model_250 = []
start = time.time()
for i in range(5):
    model.fit(train_dataset,
             epochs = 10, batch_size=10,
             validation_data = 0.0  
             )
    result = model.evaluate(test_dataset)
    model_250.append(result[1])
end = time.time()
print(end - start)

Epoch 1/10
3/3 [==============================] - 4s 1s/step - loss: 0.6943 - accuracy: 0.4318
Epoch 2/10
3/3 [==============================] - 3s 1s/step - loss: 0.6685 - accuracy: 0.7386
Epoch 3/10
3/3 [==============================] - 3s 1s/step - loss: 0.4048 - accuracy: 0.8295
Epoch 4/10
3/3 [==============================] - 4s 1s/step - loss: 0.2942 - accuracy: 0.8409
Epoch 5/10
3/3 [==============================] - 3s 1s/step - loss: 0.2578 - accuracy: 0.9205
Epoch 6/10
3/3 [==============================] - 3s 991ms/step - loss: 0.2159 - accuracy: 0.9091
Epoch 7/10
3/3 [==============================] - 3s 1s/step - loss: 0.1886 - accuracy: 0.9318
Epoch 8/10
3/3 [==============================] - 3s 1s/step - loss: 0.1423 - accuracy: 0.9659
Epoch 9/10
3/3 [==============================] - 3s 995ms/step - loss: 0.1201 - accuracy: 0.9545
Epoch 10/10
1/1 [==============================] - 0s 370ms/step - loss: 0.1816 - accuracy: 0.9091
Epoch 1/10
3/3 [========================

In [12]:
print(model_250)
print(np.mean(model_250))

[0.9090909361839294, 0.8636363744735718, 0.8181818127632141, 0.8636363744735718, 0.8636363744735718]
0.8636363744735718


In [13]:
result = model.evaluate(test_dataset)

1/1 [==============================] - 0s 259ms/step - loss: 0.3301 - accuracy: 0.8636


In [14]:
"""

#steps_per_epoch = train_imagesize/batch_size

model.fit_generator(train_dataset,
         steps_per_epoch = 250,
         epochs = 10,
         validation_data = test_dataset
       
         )
         
"""

'\n\n#steps_per_epoch = train_imagesize/batch_size\n\nmodel.fit_generator(train_dataset,\n         steps_per_epoch = 250,\n         epochs = 10,\n         validation_data = test_dataset\n       \n         )\n         \n'

In [15]:
"""

# Manually Predicting 
test_path = '../test_tumor'
Tumor=0
NoTumor=0
for i in os.listdir(test_path):
    img = image.load_img(test_path + "//" +i, target_size=(512,512,3))
    plt.imshow(img)
    plt.show()
    X = image.img_to_array(img)
    X = np.expand_dims(X, axis=0)
    images = np.vstack([X])
    val = model.predict(images)
    if val == 0:
        NoTumor +=1
        print('NoTumor')
    else:
        Tumor +=1
        print('Tumor')
print(f'Tumor: {Tumor}, NoTumor: {NoTumor}')


"""

'\n\n# Manually Predicting \ntest_path = \'../test_tumor\'\nTumor=0\nNoTumor=0\nfor i in os.listdir(test_path):\n    img = image.load_img(test_path + "//" +i, target_size=(512,512,3))\n    plt.imshow(img)\n    plt.show()\n    X = image.img_to_array(img)\n    X = np.expand_dims(X, axis=0)\n    images = np.vstack([X])\n    val = model.predict(images)\n    if val == 0:\n        NoTumor +=1\n        print(\'NoTumor\')\n    else:\n        Tumor +=1\n        print(\'Tumor\')\nprint(f\'Tumor: {Tumor}, NoTumor: {NoTumor}\')\n\n\n'

In [16]:
"""

def predictImage(filename):
    img1 = image.load_img(filename,target_size=(224,224))
    
    plt.imshow(img1)
 
    Y = image.img_to_array(img1)
    
    X = np.expand_dims(Y,axis=0)
    val = model.predict(X)
    print(val)
    if val == 1:
        
        plt.xlabel("Duck",fontsize=20)
        
    
    elif val == 0:
        
        plt.xlabel("Bird",fontsize=20)


"""

'\n\ndef predictImage(filename):\n    img1 = image.load_img(filename,target_size=(224,224))\n    \n    plt.imshow(img1)\n \n    Y = image.img_to_array(img1)\n    \n    X = np.expand_dims(Y,axis=0)\n    val = model.predict(X)\n    print(val)\n    if val == 1:\n        \n        plt.xlabel("Duck",fontsize=20)\n        \n    \n    elif val == 0:\n        \n        plt.xlabel("Bird",fontsize=20)\n\n\n'